# QuickStart

_ActionWeaver is an AI application framework that puts function-calling as a first-class feature_.

---

This demonstrates how to seamlessly integrate any Python function into your LLM application using ActionWeaver and either the Azure or OpenAI API.
 

## Use ActionWeaver and OpenAI API

In [72]:
from actionweaver.llms.openai.tools.tokens import TokenUsageTracker
from actionweaver.llms import patch
from openai import OpenAI

openai_client = patch(OpenAI())



<function actionweaver.llms.patch.patch(client: Union[openai.OpenAI, openai.AsyncOpenAI, openai.lib.azure.AsyncAzureOpenAI, openai.lib.azure.AzureOpenAI])>

In [38]:
from actionweaver import action

@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


@action(name="GetWeather", stop=False)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [44]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}
  ]


response = openai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
    actions = [get_current_weather],
    stream=False, 
    logger=logger,
    token_usage_tracker = TokenUsageTracker(500),
)

response

Getting current weather
Getting current weather


"The current weather in San Francisco is 72°F. However, I couldn't retrieve the current weather for Beijing."

## Use ActionWeaver and Azure OpenAI Service

In [52]:
import os
from openai import AzureOpenAI

azure_client = patch(AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview"
))

In [48]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}
  ]


response = azure_client.chat.completions.create(
  model="gpt-35-turbo-0613-16k",
  messages=messages,
  stream=False,
  actions = [get_current_weather],
  token_usage_tracker = TokenUsageTracker(500),
  logger=logger
)

response

Getting current weather
Getting current weather


"The current weather in San Francisco is 72 degrees Fahrenheit. However, I couldn't find the current weather temperature for Beijing."

### Force execute an action

In [54]:
get_current_time.invoke(openai_client, messages=[{"role": "user", "content": "what time"}], model="gpt-3.5-turbo", stream=False, force=False)


Getting current time...


'The current time is 23:21 (11:21 PM).'

In [56]:
get_current_time.invoke(azure_client, messages=[{"role": "user", "content": "what time"}], model="gpt-35-turbo-0613-16k", stream=False, force=False)


Getting current time...


'The current time is 23:21.'

### Stop the Action in the loop

Every action comes with a stop argument, which is set to False by default, if True this means that the LLM will immediately return the function's output if chosen, but this also restricts the LLM from making multiple function calls. For instance, if asked about the weather in NYC and San Francisco, the model would invoke two separate functions sequentially for each city. However, with `stop=True`, this process is interrupted once the first function returns weather information for either NYC or San Francisco, depending on which city it queries first.

In [67]:
get_current_weather.stop = True

In [68]:
get_current_weather.invoke(openai_client, messages=[{"role": "user", "content": "what weather is San Francisco"}], model="gpt-3.5-turbo", stream=False, force=True)


Getting current weather


'{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'